In [2]:
import networkx as nx
from collections import defaultdict
import matplotlib.pyplot as plt 
import pandas as pd
import folium
from folium import plugins
import ipywidgets

In [3]:
G = nx.Graph()

# NODES of GRAPH Network

In [4]:
with open(r'USA-road-d.CAL.co','r') as f1:
    for line in f1:
        if line[0] == 'v':
            n,la,lo = list(map(int, line[2:].split()))
            G.add_node(n,latitude = lo/1000000,longitude = la/1000000)

In [5]:
G.nodes[5]

{'latitude': 34.042614, 'longitude': -114.3473}

In [6]:
nx.info(G)

'Name: \nType: Graph\nNumber of nodes: 1890815\nNumber of edges: 0\nAverage degree:   0.0000'

# Edges (Distance) of Graph Network

In [7]:
adj = defaultdict(set)
with open(r'USA-road-d.CAL.gr','r') as f:
    for line in f:
        if line[0] == 'a':
            n1,n2, d =  list(map(int, line[2:].split()))
            G.add_edge(n1,n2,distance = d,weight = 1)
            adj[n1].add(n2)
            adj[n2].add(n1)

In [8]:
nx.info(G)

'Name: \nType: Graph\nNumber of nodes: 1890815\nNumber of edges: 2315222\nAverage degree:   2.4489'

In [9]:
G.edges[5,6]

{'distance': 18133, 'weight': 1}

# Edges (Time) of Graph Network

In [10]:
with open(r'USA-road-t.CAL.gr','r') as f2:
    for line in f2:
        if line[0] == 'a':
            n1,n2, t =  list(map(int, line[2:].split()))
            G.add_edge(n1,n2,time = t)

In [11]:
nx.info(G)

'Name: \nType: Graph\nNumber of nodes: 1890815\nNumber of edges: 2315222\nAverage degree:   2.4489'

In [12]:
G.edges[5,6]

{'distance': 18133, 'weight': 1, 'time': 45332}

In [13]:
nx.has_path(G, source=5, target=6)

True

In [14]:
nx.dijkstra_path(G, source=2244, target=1050368)

[2244, 5, 6, 1050368]

In [15]:
# Making Nodes in Network into Dataframe
#NodesDataframe = pd.DataFrame([G.nodes[i] for i in range(1,len(G.nodes))])
#NodesDataframe.head()

# Map Function

In [16]:
# Map

# V is the input node or start node

def map(nodelst):
      
    pos = G.nodes[nodelst[0]]# for the start point
    
    # creating the Map focusing on the start point

    vismap = folium.Map(location=[pos['latitude'], pos['longitude']], zoom_start=10)
    
    #Layers Map
    
    folium.raster_layers.TileLayer('Open Street Map').add_to(vismap)
    folium.raster_layers.TileLayer('Stamen Terrain').add_to(vismap)
    
    #adding lcontrol on map
    folium.LayerControl().add_to(vismap)
    
    #minimap
    
    
    # plugin for mini map
    visminimap = plugins.MiniMap(toggle_display=True)

    # add minimap to map
    vismap.add_child(visminimap)

    # add scroll zoom toggler to map
    plugins.ScrollZoomToggler().add_to(vismap)

    # creating a marker of HOme on the start point
    
    folium.Marker(location=[(pos['latitude']),(pos['longitude'])],
                  icon=folium.Icon(color='red', icon='home'), popup = (nodelst[0])).add_to(vismap)

    # creating a marker on the rest of the point
    
    for i in range (len(nodelst)-1):
        pos = (G.nodes[nodelst[i+1]])
        folium.Marker(location=[(pos['latitude']),(pos['longitude'])],popup = (nodelst[i+1])).add_to(vismap)
    
    return vismap

# Map Route Function

In [17]:
# a is the list of nodes
# map_name is the map with nodes already generated


def map_routes(nodelst,map_name):
    cordlst = 0
    cordlst = []
    # making list of all coordinates of the NODES avaialble in a
    for i in nodelst:
        cordlst.append(list(G.nodes[i].values())) 
    plugins.AntPath(cordlst).add_to(map_name)
       
    return map_name

# Function 1a

In [18]:
# v is the input Node
# P is the type of weight distance or time
# d is the thrushhold
# neigh [] is a list of neighbour

def fun1(v,p,d,neigh = []):
    neigh += [v]
    adjs = adj[v]
    for node in adjs:
        if G[v][node][p] <= d and node not in neigh:
            new_d = d-G[v][node][p]
            neigh = fun1(node,p,new_d,neigh)
    return neigh

# Function 1b

In [19]:
import heapq as hp
def neighbors(v,w,d,graph = G, adjacent = adj):
    # In the visited dictionary I insert node and minimum weight to reach it.
    # Usually with the set of nodes visited in the djikstra algorithm we mean all the nodes that are visited 
    # and for which the weight is no longer changed but here I include all the ones that can be changed 
    # but at the end of the while cycle at each node will be assigned the final weight
    visited = {v: 0}
    F = []
    # F is a heap where I keep all the nodes that can extend the tree of the shortest paths, 
    # I use a minheap because I have to extract the minimum and the extraction of the minimum happens in O (1) time
    # even if then reassigning to the root the minimum value uses O (logn) time is reasonable.
    hp.heapify(F)
    # the input node v is the starting node
    current_node = v
    while True:
        # I take all the nodes adjacent to v
        adjacents = adjacent[current_node]
        # I take the weight of the current node from the visited dictionary
        weight_to_current_node = visited[current_node]
        for node in adjacents:
            # I take the weights of all the adjacent ones calculating it as the sum of the weight of the adjacent node 
            # and the weight of the edge that connects them according to the weight function passed in input.
            weight = graph[current_node][node][w] + weight_to_current_node
            #here I filter the data, I take only those at a distance less than d.
            #I insert only the nodes that can extend the tree of the shortest paths but weigh less than the treshold d
            #in visited and in the border F.
            if weight <= d:
                # if the node is the first time I reach it simply add to the visited and at the border 
                #the node with its weight calculated as the sum of the weight of the node that allowed it to be reached 
                #and the weight of the arch that connects them
                if node not in visited:
                    visited[node] = weight
                    hp.heappush(F,(weight,node))
                else:
                    #instead if the node has already been visited and the distance assigned to the node
                    #is greater than that given by the sum of the weight of the new node that allowed us to reach 
                    #it and the edge that connects them then the weight associated with the node is updated
                    #with this last.And a new weight node tuple is inserted into the heap.
                    #Here is the only weakness of our algorithm (at least the ones I saw)
                    #inasmuch as having more weight node pairs it will be more iterations than necessary, 
                    #this however does not impact on the correctness of the result because I from the heap simply
                    #take the node while I recover the weight from the visited dictionary where it is updated
                    #with the best value, at time level a little slows down to do more iterations but a reasonable 
                    #time having done many tests I also tried to eliminate this problem but the only way 
                    #I found was to go over a list to delete the tuple and insert the new tuple to 
                    #then recreate the heap but doing various tests these operations were heavier than doing 
                    #some more iteration.
                    current_shortest_weight = visited[node]
                    if current_shortest_weight > weight:
                        visited[node] = weight
                        hp.heappush(F,(weight,node))
        # I extract the vertices until the border is empty, this means that in my dictionary 
        #I will have all the nodes at a distance less than d, in fact they have the list of keys of visited
        try:
            current_node = hp.heappop(F)[1]
        except:
            break
    neighbors = list(visited.keys())
    return neighbors

# Function 3 part (a)- shortest path

In [31]:
def dijkstra_modified(v,end,p,graph = G,adjacent = adj):
    # v is the start node
    # end is the destination node
    # Also here as in the functionality 1 I use djikstra with small changes here instead of calculating 
    # the neighbors of the starting node I calculate the minimum path between the starting node and the arrival node.
    # I create a dictionary of the visited ones but besides the weight I save the predecessor
    # that then it will be useful to me to reconstruct the path
    visited = {v: (None, 0)}
    F = []
    hp.heapify(F)
    current_node = v
    #the core part of the code is the same with the exception that as the output of the while 
    #I have reached the destination node
    while current_node != end:
        adjacentes = adjacent[current_node]
        weight_to_current_node = visited[current_node][1]
        for node in adjacentes:
            weight = graph[current_node][node][p] + weight_to_current_node
            if node not in visited:
                visited[node] = (current_node, weight)
                hp.heappush(F,(weight,node))
            else:
                current_shortest_weight = visited[node][1]
                if current_shortest_weight > weight:
                    visited[node] = (current_node, weight)
                    hp.heappush(F,(weight,node))
        #also here I check if the border is empty, if it's empty and we haven't left yet while it means that 
        #I checked all the nodes without having reached the destination node 
        #so I can say that there is no path between the starting node and the one of destination
        if not F:
            return "Route Not Possible"
        current_node = hp.heappop(F)[1]
    # Work backwards between the destinations visited up to the node that has no parent set to None, then up to the
    #first node. I start from the current node because after the while the current node is the destination node
    path = []
    while current_node is not None:
        path.append(current_node)
        next_node = visited[current_node][0]
        current_node = next_node
     # having added the nodes in the list from the last one obviously before returning it I reverse it
    path.reverse()
    return path

# Function 3 part (b)- ordered Path

In [26]:
def order_walk(v,nodes,p):
    path = dijkstra(v,nodes[0],p)
    #here I check if the output of my function is a string if it is a string 
    #it means that there is no path between the two nodes and I simply return it.
    if type(path) == str:
        return path
    for i in range(1,len(nodes)):
        #concatenate the paths between the pairs of nodes to create the ordered walk
        path1 = dijkstra(nodes[i-1],nodes[i],p)
        #I repeat the check if there is a path or not
        if type(path1) == str:
            return path1
        else:
            path += path1[1:]
    #return final walk
    return path

In [18]:
start = input(" Do you want to Start the FN :")

 Do you want to Start the FN :y


# Visualization F1a - Front End

In [7]:
v = (int(input('Enter the node number: ')))
p = (input('Enter the type of weight, Either distance or time: '))
d = int(input('Enter the thrushhold for the type of weight: '))
nodelst = fun1(v,p,d)
nodelst
map(nodelst).save('F1amap.html')
#m.save('F1map.html')

'Name: \nType: Graph\nNumber of nodes: 1890815\nNumber of edges: 2315222\nAverage degree:   2.4489'

# Visualization F1b - Front End

In [23]:
v = (int(input('Enter the node number: ')))
p = (input('Enter the type of weight, Either distance or time: '))
d = int(input('Enter the thrushhold for the type of weight: '))
nodelst = 0
nodelst = neighbors(v,p,d,graph = G, adjacent = adj)
nodelst
map(nodelst).save('F1bmap.html')
#m.save('F1map.html')

Enter the node number: 6
Enter the type of weight, Either distance or time: time
Enter the thrushhold for the type of weight: 100000


 # Visualization F3 - Front End

In [36]:
s = int(input("Enter the Home Node: "))
k = int(input("Enter the departure Node: "))
p = (input('Enter the type of weight, Either distance or time: '))
nodelst = 0
nodelst = dijkstra_modified(s,k,p,graph = G,adjacent = adj)
fpath = order_walk(s,nodelst,p)
#nodelst = nx.dijkstra_path(G, source=s, target=t)
c = map(fpath)
map_routes(fpath,c).save('F3map.html')

Enter the Home Node: 6
Enter the departure Node: 8
Enter the type of weight, Either distance or time: time
